In [ ]:
from datasets import load_dataset, DatasetDict, Dataset
import pandas as pd
import expressoes as exp
import re
from tqdm.auto import tqdm
from pandarallel import pandarallel
from cleaner import Cleaner, Corretor, executaLimpeza

### Continuação dos 14 indícios a partir da base com back_translate

In [ ]:
dados = load_dataset('tcepi/bidCorpus_14_indicios', token=token)
dados

In [ ]:
dados_teste = pd.DataFrame(dados['test'])
dados_teste

In [ ]:
dados_csv = pd.DataFrame(dados['train'])
dados_csv

In [ ]:
amostras_corretas = []
for indice in tqdm(dados_csv.index):
    for name_expression, expression in exp.expressoes.items():
        if re.search(expression, dados_csv['text'][indice]):
            amostras_corretas.append(dados_csv.loc[indice])
            break
df_com_indicios = pd.DataFrame(amostras_corretas)
df_com_indicios

In [ ]:
df_com_indicios.dropna(subset=['text'], inplace=True)
df_com_indicios.drop_duplicates(subset=['text'], inplace=True)
df_com_indicios.reset_index(drop=True, inplace=True)
df_com_indicios

In [ ]:
indicios_antes = dados_csv.drop(columns=['text']).sum()
indicios_antes

In [ ]:
df_com_indicios.drop(columns=['text']).sum()

In [ ]:
df_com_indicios.drop(columns=['contratacao_visto_registro_profissional'], inplace=True)
df_com_indicios

In [ ]:
# Reduzindo as duas classes majoritárias
indices_garantia_cs_ou_pl = []
indices_temp_exp_profissional = []
for indice in tqdm(df_com_indicios.index):
    if df_com_indicios['garantia_cs_ou_pl'][indice] == 1 and df_com_indicios.iloc[indice, 1:].sum() == 1:
        indices_garantia_cs_ou_pl.append(indice)
    elif df_com_indicios['temp_exp_profissional'][indice] == 1 and df_com_indicios.iloc[indice, 1:].sum() == 1:
        indices_temp_exp_profissional.append(indice)

In [ ]:
len(indices_garantia_cs_ou_pl), len(indices_temp_exp_profissional)

In [ ]:
import random

indices_garantia_cs_ou_pl_drop = random.sample(indices_garantia_cs_ou_pl, 2100)
indices_temp_exp_profissional_drop = random.sample(indices_temp_exp_profissional, 800)
print(len(indices_garantia_cs_ou_pl_drop), len(indices_temp_exp_profissional_drop))

In [ ]:
df_com_indicios.drop(index=indices_garantia_cs_ou_pl_drop, inplace=True)
df_com_indicios.drop(index=indices_temp_exp_profissional_drop, inplace=True)
df_com_indicios.reset_index(drop=True, inplace=True)
df_com_indicios

In [ ]:
indicios_presentes

In [ ]:
df_com_indicios.drop(columns=['text']).sum()

In [ ]:
df_com_indicios.to_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsWeak/DataAugmentation/BackTranslateRevisado/df_com_indicios_balanceado.csv', index=False)

#### Limpeza do dataset de indícios

In [ ]:
dados = pd.read_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsWeak/DataAugmentation/BackTranslateRevisado/df_com_indicios_balanceado.csv')
dados

In [ ]:
dataset_limpo = executaLimpeza(dados, 'text', False, False)
dataset_limpo   

In [ ]:
df_com_indicios.columns

In [ ]:
dados.to_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsWeak/DataAugmentation/BackTranslateRevisado/df_com_indicios_balanceado_limpo.csv', index=False)

### Adição de amostras de outros datasets

#### Dataset Gold

In [ ]:
dados_gold = load_dataset("csv", data_files="/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsRaw/dataset_indicios_gold_raw.csv")
dados_gold = pd.DataFrame(dados_gold['train'])
dados_gold

In [ ]:
colunas_desejadas = [
    'text', 'garantia_cs_ou_pl', 'carta_credenciamento',
    'filiacao_abav_iata', 'temp_exp_profissional', 'atestado_regularizacao',
    'atestado_carga', 'certidao_protesto', 'certificado_boas_praticas',
    'comprovante_localizacao', 'idoneidade_financeira', 'integralizado',
    'licenca_ambiental', 'n_min_max_limitacao_atestados'
]

# Adicionar as colunas que estão faltando e garantir que todas as colunas estejam na sequência correta
for coluna in colunas_desejadas:
    if coluna not in dados_gold.columns:
        dados_gold[coluna] = 0

# Reordenar as colunas de acordo com colunas_desejadas
dados_gold = dados_gold[colunas_desejadas]
dados_gold

In [ ]:
for indice in tqdm(dados_gold.index):
    for column in dados_gold.columns[1:7]:
        if re.search(exp.expressoes[column], dados_gold['text'][indice]):
            dados_gold.loc[indice, column] = 1
dados_gold

In [ ]:
dados_gold.drop(columns=['text']).sum()

In [ ]:
dados_gold = executaLimpeza(dados_gold, 'text', False, False)
dados_gold

In [ ]:
dados_gold.to_csv("/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsWeak/DataAugmentation/BackTranslateRevisado/dados_gold_limpo_14_indicios.csv", index=False)

#### Dataset weak sem indícios

In [ ]:
dados_raw = load_dataset("tcepi/bidCorpus_raw", "bid_corpus_raw", token=token)
dados_raw = pd.DataFrame(dados_raw['train']['HABILITACAO'], columns=['text'])
dados_raw.drop_duplicates(subset=['text'], inplace=True)
dados_raw.dropna(subset=['text'], inplace=True)
dados_raw.reset_index(drop=True, inplace=True)
dados_raw

In [ ]:
columns_to_add = [
    'garantia_cs_ou_pl', 'carta_credenciamento', 'filiacao_abav_iata',
    'temp_exp_profissional',
    'atestado_regularizacao', 'atestado_carga', 'certidao_protesto',
    'certificado_boas_praticas', 'comprovante_localizacao',
    'idoneidade_financeira', 'integralizado', 'licenca_ambiental',
    'n_min_max_limitacao_atestados'
]

for column in columns_to_add:
    dados_raw[column] = 0

dados_raw

In [ ]:
dados_raw = dados_raw.sample(n=2000)
dados_raw.reset_index(drop=True, inplace=True)
dados_raw

In [ ]:
for indice in tqdm(dados_raw.index):
    for column in dados_raw.columns[1:]:
        if re.search(exp.expressoes[column], dados_raw['text'][indice]):
            dados_raw.loc[indice, column] = 1
dados_raw

In [ ]:
rows_to_add = []
for indice in tqdm(dados_raw.index):
    if dados_raw.iloc[indice, 1:].sum() == 0:
        rows_to_add.append(dados_raw.loc[indice])
df_add = pd.DataFrame(rows_to_add)
df_add

In [ ]:
df_add.drop_duplicates(subset=['text'], inplace=True)
df_add.dropna(subset=['text'], inplace=True)
df_add = df_add.sample(n=1000)
df_add.reset_index(drop=True, inplace=True)
df_add

In [ ]:
df_add = executaLimpeza(df_add, 'text', False, False)
df_add

In [ ]:
df_add.to_csv("/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsWeak/DataAugmentation/BackTranslateRevisado/df_sem_indicios.csv", index=False)

### Junção dos datasets (gold, data augmentation, sem indicios)

In [ ]:
gold = pd.read_csv("/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsWeak/DataAugmentation/BackTranslateRevisado/dados_gold_limpo_14_indicios.csv")
balanceado = pd.read_csv("/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsWeak/DataAugmentation/BackTranslateRevisado/df_com_indicios_balanceado_limpo.csv")
sem_indicios = pd.read_csv("/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsWeak/DataAugmentation/BackTranslateRevisado/df_sem_indicios.csv")

In [ ]:
dados = pd.concat([gold, balanceado, sem_indicios])
dados.drop_duplicates(subset=['text'], inplace=True)
dados.dropna(subset=['text'], inplace=True)
dados.reset_index(drop=True, inplace=True)
dados

In [ ]:
dados.drop(columns=['text']).sum()

In [ ]:
dados_teste

In [ ]:
dados.to_csv("/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsWeak/DataAugmentation/BackTranslateRevisado/df_unificado_final.csv", index=False)

### Convertendo em dataset_dict e mandando pro huggingface

In [ ]:
train = pd.read_csv("/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsWeak/DataAugmentation/BackTranslateRevisado/df_unificado_final.csv")
teste = load_dataset("tcepi/bidCorpus_14_indicios", token=token)['test']

In [ ]:
teste = pd.DataFrame(teste)
teste

In [ ]:
teste = executaLimpeza(teste, 'text', False, False)
teste

In [ ]:
indices_drop = []
for indice in tqdm(train.index):
    if not isinstance(train['text'][indice], str):
        indices_drop.append(indice)
train.drop(index=indices_drop, inplace=True)

train.dropna(subset=['text'], inplace=True)
train.drop_duplicates(subset=['text'], inplace=True)
train.reset_index(drop=True, inplace=True)

indices_drop = []
for indice in tqdm(teste.index):
    if not isinstance(teste['text'][indice], str):
        indices_drop.append(indice)
teste.drop(index=indices_drop, inplace=True)

teste.dropna(subset=['text'], inplace=True)
teste.drop_duplicates(subset=['text'], inplace=True)
teste.reset_index(drop=True, inplace=True)

print(train.shape, teste.shape)

In [ ]:
train.drop(columns=['text']).sum()

In [ ]:
teste.drop(columns=['contratacao_visto_registro_profissional'], inplace=True)
teste

In [ ]:
train = Dataset.from_pandas(train)
teste = Dataset.from_pandas(teste)

In [ ]:
dict_dataset = DatasetDict({
    'train': train,
    'test': teste,
})
dict_dataset

In [ ]:
dict_dataset.push_to_hub("tcepi/bidCorpus_14_indicios_revisado", token="")

### Buscando a proporção de indícios no dataset original

In [ ]:
dados_raw = load_dataset("tcepi/bidCorpus_raw", "bid_corpus_raw", token=token)
dados_raw = pd.DataFrame(dados_raw['train']['HABILITACAO'], columns=['text'])
dados_raw.drop_duplicates(subset=['text'], inplace=True)
dados_raw.dropna(subset=['text'], inplace=True)
dados_raw.reset_index(drop=True, inplace=True)
dados_raw

In [ ]:
columns_to_add = [
    'garantia_cs_ou_pl', 'carta_credenciamento', 'filiacao_abav_iata',
    'temp_exp_profissional',
    'atestado_regularizacao', 'atestado_carga', 'certidao_protesto',
    'certificado_boas_praticas', 'comprovante_localizacao',
    'idoneidade_financeira', 'integralizado', 'licenca_ambiental',
    'n_min_max_limitacao_atestados'
]

for column in columns_to_add:
    dados_raw[column] = 0

dados_raw

In [ ]:
def process_row(row):
    for name_expression, expression in exp.expressoes.items():
        if re.search(expression, row['text']):
            row[name_expression] = 1
    return row

In [ ]:
pandarallel.initialize(progress_bar=True)

In [ ]:
dados_raw = dados_raw.parallel_apply(process_row, axis=1)
dados_raw

In [ ]:
dados_raw.drop(columns=['text']).sum()

In [ ]:
lista_sem_indicios = []
for indice in tqdm(dados_raw.index):
    if dados_raw.iloc[indice, 1:].sum() == 0:
        lista_sem_indicios.append(indice)

In [ ]:
len(lista_sem_indicios)

In [ ]:
print(len(lista_sem_indicios)/len(dados_raw))

In [ ]:
df_sem_indicios = pd.DataFrame(dados_raw.loc[lista_sem_indicios])
df_sem_indicios.reset_index(drop=True, inplace=True)
df_sem_indicios

In [ ]:
sem_indicios_df_total = df_sem_indicios.sample(n=13000)
sem_indicios_df_total.reset_index(drop=True, inplace=True)
sem_indicios_df_total

In [ ]:
sem_indicios_df_total = executaLimpeza(sem_indicios_df_total, 'text', False, False)
sem_indicios_df_total

In [ ]:
sem_indicios_df_total.to_csv("/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsWeak/DataAugmentation/BackTranslateRevisado/df_sem_indicios.csv", index=False)

### Juntando com o df final

In [ ]:
dados_final = pd.read_csv("/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsWeak/DataAugmentation/BackTranslateRevisado/df_unificado_final.csv")
dados_final

In [ ]:
dados = pd.concat([dados_final, sem_indicios_df_total])
dados.drop_duplicates(subset=['text'], inplace=True)
dados.dropna(subset=['text'], inplace=True)
dados.reset_index(drop=True, inplace=True)
dados

In [ ]:
dados.to_csv("/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsWeak/DataAugmentation/BackTranslateRevisado/df_unificado_final.csv", index=False)